# PROMPT RESULT ANALYSIS

In [ ]:
import json
from utils import plan_helper

pred_dict = json.load(open('result/alfred/prompt20/roberta_penalty/valid_seen-plan_13.json', 'r'))
num_plan = []
for goal in pred_dict:
    plans = pred_dict[goal]['plan']
    num_plan.append(plan_helper.count_plans(plans))
    
import matplotlib.pyplot as plt
nBin = (max(num_plan)-min(num_plan)+1)
counts, edges, bars = plt.hist(num_plan, bins=nBin)
plt.bar_label(bars)
plt.show()

In [ ]:
import json
import os
import re

def preprocess(s):
    # remove escape sequence
    s = s.strip()
    if "\n" in s:
        s = s.split("\n")[0]
    elif "\b" in s:
        s = s.strip("\b")
    # CounterTop -> Counter Top
    if not any(_s.isupper() for _s in s):
        # lower
        return s.lower()
    l = re.findall('[A-Z][^A-Z]*', s)
    ls = []
    left = s.split(l[0])[0]
    for _l in l:
        if len(_l) == 1:
            left += _l
        else:
            _l = left+_l
            ls.extend(_l.split())
            left = ''
    # lower
    return ' '.join(ls).lower().replace('.', '')

goal2plan = json.load(open('result/alfred/prompt20/roberta_penalty/valid_seen-plan_8.json', 'r'))
goal2sentences = json.load(open('result/alfred/roberta/valid_seen-sentence@20.json', 'r'))

# (Acc, Recall, F1, goal)
success = []
fail = []
kAcc = 0

for ep in os.listdir('data/alfred_data/json_2.1.0/valid_seen'):
    for trial in os.listdir(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep)):
        traj_data = json.load(open(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep, trial, 'traj_data.json')))
        anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
        gt = [action['discrete_action'] for action in traj_data['plan']['high_pddl'] if action['discrete_action']['action'] not in ['GotoLocation', 'NoOp']]
        for goal in anns:
            try:
                preds = goal2plan[goal]['plan']
            except:
                continue
            pred = preds[0]
            tp = 0
            for action in pred:
                temp = []
                for arg in action['args']:
                    temp.append(arg.lower())
                action['args'] = temp
                if action in gt:
                    tp += 1
            acc = tp/len(pred)
            recall = tp/len(gt)
            try:
                f1 = 2/(1/acc+1/recall)
            except ZeroDivisionError:
                f1 = 0
            if acc == 1 and recall == 1:
                success.append((acc, recall, f1, goal))
            else:
                fail.append((acc, recall, f1, goal))
            result = []
            for j, pred in enumerate(preds):
                if len(pred) != len(gt):
                    result.append(False)
                    continue
                for i, action in enumerate(pred):
                    temp = []
                    for arg in action['args']:
                        temp.append(arg.lower())
                    action['args'] = temp
                    if gt[i] != action:
                        result.append(False)
                        break
                if len(result) != j+1:
                    result.append(True)
            if any(result):
                kAcc += 1
                # print(gt)
                # print(preds)
print('@kAcc: %.2f'%(kAcc/40))
print(kAcc)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

success_xs = np.array([1]*len(success))
success_ys = np.array([goal2plan[s[-1]]['prompt_logp'] for s in success]) 

fail_xs = []
fail_ys = []
for ex in fail:
    fail_xs.append(ex[0])
    fail_ys.append(goal2plan[ex[-1]]['prompt_logp'])

plt.figure(figsize=(12, 3))
acc_plot = plt.subplot(1, 3, 1)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('ACC')
plt.xlabel('gpt logp')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[1])
plt.subplot(1, 3, 2)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('Recall')
plt.xlabel('gpt logp')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[2])
plt.subplot(1, 3, 3)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('F1')
plt.xlabel('gpt logp')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

success_xs = np.array([1]*len(success))
success_ys = np.array([np.mean(np.log(np.array(goal2plan[s[-1]]['trans_cossim']))) for s in success]) 

fail_xs = []
fail_ys = []
for ex in fail:
    fail_xs.append(ex[0])
    fail_ys.append(np.mean(np.log(np.array(goal2plan[ex[-1]]['trans_cossim']))))

plt.figure(figsize=(12, 3))
acc_plot = plt.subplot(1, 3, 1)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('ACC')
plt.xlabel('avg(log(cos_sim))')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[1])
plt.subplot(1, 3, 2)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('Recall')
plt.xlabel('avg(log(cos_sim))')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[2])
plt.subplot(1, 3, 3)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('F1')
plt.xlabel('avg(log(cos_sim))')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

success_xs = np.array([1]*len(success))
success_ys = np.array([min(goal2plan[s[-1]]['trans_cossim']) for s in success]) 

fail_xs = []
fail_ys = []
for ex in fail:
    fail_xs.append(ex[0])
    fail_ys.append(min(goal2plan[ex[-1]]['trans_cossim']))

plt.figure(figsize=(12, 3))
acc_plot = plt.subplot(1, 3, 1)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('ACC')
plt.xlabel('min(cos_sim)')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[1])
plt.subplot(1, 3, 2)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('Recall')
plt.xlabel('min(cos_sim)')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[2])
plt.subplot(1, 3, 3)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('F1')
plt.xlabel('min(cos_sim)')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
# 100의 표준편차 점수

success_xs = np.array([])
success_ys = np.array([])
for s in success:
    success_xs = np.append(success_xs, 1)
    scores = np.array(goal2sentences[s[-1]]['score'])
    success_ys = np.append(success_ys, np.std(scores))

fail_xs = []
fail_ys = []
for ex in fail:
    fail_xs = np.append(fail_xs, ex[0])
    scores = np.array(goal2sentences[ex[-1]]['score'])
    fail_ys = np.append(fail_ys, np.std(scores))

plt.figure(figsize=(12, 3))
acc_plot = plt.subplot(1, 3, 1)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('ACC')
plt.xlabel('std cos_sim @k')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[1])
plt.subplot(1, 3, 2)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('Recall')
plt.xlabel('std cos_sim @k')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[2])
plt.subplot(1, 3, 3)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('F1')
plt.xlabel('std cos_sim @k')

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.stats import norm
# 100의 표준편차 점수

success_xs = np.array([])
success_ys = np.array([])
for s in success:
    success_xs = np.append(success_xs, 1)
    scores = np.array(goal2sentences[s[-1]]['score'])
    z = (1-np.mean(scores))/np.std(scores)
    success_ys = np.append(success_ys, (1-norm.cdf(z)))

fail_xs = []
fail_ys = []
for ex in fail:
    fail_xs = np.append(fail_xs, ex[0])
    scores = np.array(goal2sentences[ex[-1]]['score'])
    z = (1-np.mean(scores))/np.std(scores)
    fail_ys = np.append(fail_ys, 1-norm.cdf(z))

plt.figure(figsize=(12, 3))
acc_plot = plt.subplot(1, 3, 1)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('ACC')
plt.xlabel('normal dist P(1 < cos_sim @k)')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[1])
plt.subplot(1, 3, 2)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('Recall')
plt.xlabel('normal dist P(1 < cos_sim @k)')

fail_xs = []
for ex in fail:
    fail_xs.append(ex[2])
plt.subplot(1, 3, 3)
plt.plot(success_ys, success_xs, 'ro')
plt.plot(fail_ys, fail_xs, 'bo')
plt.ylabel('F1')
plt.xlabel('normal dist P(1 < cos_sim @k)')

plt.show()

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('valid seen')
k = [1, 5, 10, 15, 20, 50, 100]
acc = [57.87, 75.59, 79.68, 82.16, 83.34, 87.86, 90.09]
film_full = 77.52
film = 67.20
hier1 = 86.01
hier2 = 87.56

plt.ylim(50, 95)
plt.plot(k, acc, 'bo', label='RoBERTa')
plt.xticks(k)
for x, y in zip(k, acc):
    plt.annotate(y, (x, y), textcoords='offset points', xytext=(0, 10), ha='center')
plt.xlabel('k (# of samples)')
plt.ylabel('@k acc')
plt.hlines(film_full, min(k), max(k), label='FILM(low+high)', colors='coral', linestyles='dashed')
plt.hlines(film, min(k), max(k), label='FILM(high only)', colors='coral')
plt.hlines(hier1, min(k), max(k), label='Hierarchical(prev)', colors='lightgreen', linestyles='dashed')
plt.hlines(hier2, min(k), max(k), label='Hierarchical(updated)', colors='green', linestyles='dashed')
plt.legend(loc='lower right')

plt.subplot(1, 2, 2)
plt.title('valid unseen')
k = [1, 5, 10, 15, 20, 50, 100]
acc = [54.41, 66.67, 70.88, 72.46, 73.91, 78.13, 79.45]
film_full = 73.34
film = 66.45
hier1 = 72.29
hier2 = 85.02

plt.ylim(50, 95)
plt.plot(k, acc, 'bo', label='RoBERTa')
plt.xticks(k)
for x, y in zip(k, acc):
    plt.annotate(y, (x, y), textcoords='offset points', xytext=(0, 10), ha='center')
plt.xlabel('k (# of samples)')
plt.ylabel('@k acc')
plt.hlines(film_full, min(k), max(k), label='FILM(low+high)', colors='coral', linestyles='dashed')
plt.hlines(film, min(k), max(k), label='FILM(high only)', colors='coral')
plt.hlines(hier1, min(k), max(k), label='Hierarchical(prev)', colors='lightgreen', linestyles='dashed')
plt.hlines(hier2, min(k), max(k), label='Hierarchical(updated)', colors='green', linestyles='dashed')
plt.legend(loc='lower right')

plt.show()

In [ ]:
import os
import re
from utils import plan_helper
import time
from tqdm import tqdm
import json
from multiprocessing import Pool
from utils import pre_cal_param_acc

OPENABLE_CLASS_LIST = ['Fridge', 'Cabinet', 'Microwave', 'Drawer', 'Safe']

data = json.load(open('result/alfred/roberta/valid_seen-sentence@100.json', 'r'))
# data = json.load(open('data/film/val_seen_full-plan.json', 'r'))
suc, cnt = 0, 0

def is_gtPlan_in_prompt(goal, sim_goals):
    train_plan_dict = json.load(open('data/train_tripletPlan.json', 'r'))
    for ep in os.listdir('data/alfred_data/json_2.1.0/valid_seen'):
        for trial in os.listdir(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep)):
            traj_data = json.load(open(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep, trial, 'traj_data.json'), 'r'))
            anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
            if goal not in anns:
                continue
            gt_plan = plan_helper.make_triplet_plan(traj_data)
    
    for action in gt_plan:
        if action[0] in ['PickupObject', 'SliceObject'] and action[2] in OPENABLE_CLASS_LIST:
            return True
    return False

def is_film_right(goal, plan):
    for ep in os.listdir('data/alfred_data/json_2.1.0/valid_seen'):
        for trial in os.listdir(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep)):
            traj_data = json.load(open(os.path.join('data/alfred_data/json_2.1.0/valid_seen', ep, trial, 'traj_data.json'), 'r'))
            anns = [pre_cal_param_acc.film_preprocess(ann['task_desc']) for ann in traj_data['turk_annotations']['anns']]
            if goal not in anns:
                continue
            gt_plan = plan_helper.make_triplet_plan(traj_data)
    # ignore type of knife and lamp
    toggle = False
    slice = False
    for action in gt_plan:
        if action[0] == 'SliceObject':
            slice = True
        if action[0] == 'ToggleObject':
            toggle = True

    if len(plan) != len(gt_plan):
        return False
    
    for i in range(len(gt_plan)):
        # action 다름
        if gt_plan[i][0] != plan[i]['action']:
            return False
        # put action
        if gt_plan[i][0] == 'PutObject':
            if slice and 'knife' in gt_plan[i][1].lower() and 'knife' in plan[i]['args'][0].lower():
                continue
            if gt_plan[i][2].lower() != plan[i]["args"][1].lower():
                return False
        # pick action or slice action
        elif gt_plan[i][0] in ['PickupObject', 'SliceObject']:
            # if not (gt_plan[i][2] == '' or gt_plan[i][2] not in OPENABLE_CLASS_LIST):
            #     return False
            # if slice and 'knife' in gt_plan[i][1].lower() and 'knife' in plan[i]['args'][0].lower() and \
            #     (gt_plan[i][2] == '' or gt_plan[i][2] not in OPENABLE_CLASS_LIST):
            #     continue
            # if not slice and (gt_plan[i][1] != plan[i]['args'][0] or \
            #     (gt_plan[i][2] != '' and gt_plan[i][2] in OPENABLE_CLASS_LIST)):
            #     return False
            if slice and 'knife' in gt_plan[i][1].lower() and 'knife' in plan[i]['args'][0].lower():
                continue
            if not slice and gt_plan[i][1] != plan[i]['args'][0]:
                return False
        # toggle action
        elif gt_plan[i][0] == 'ToggleObject':
            if 'lamp' in gt_plan[i][1].lower() and 'lamp' in plan[i]['args'][0].lower():
                continue
        else:
            if gt_plan[i][1] != plan[i]['args'][0]:
                return False
    return True

pool = Pool(16)
for retr in pool.starmap(is_gtPlan_in_prompt, tqdm(data.items())):
    cnt += 1
    if retr:
        suc += 1
pool.close()
pool.join()
###FOR DEBUG
# for goal, sim_goals in tqdm(data.items()):
#     is_film_right(goal, sim_goals)
        
print('%d / %d: %.2f'%(suc, cnt, suc/cnt*100))

# HuggingFace 써보기

## pipeline

자동으로 기본 모델과 preprocessing을 사용해서 inference한다.

In [ ]:
from transformers import pipeline

generator = pipeline() 
# task='', model='',  -- huggingface hub에 있는대로 가져와야 함. ex) openai/whisper-large
# task specific parameters는 task API 확인. ex) transformers.[taskname]Pipeline.call()의 파라미터
result = generator(input_data) # 여러개면 list of data

In [2]:
from transformers import pipeline

generator = pipeline(model='gpt2', device=0)
print(generator('New way to solve the problem is'))

/home/user/anaconda3/envs/language-planner/lib/python3.7/site-packages/transformers/generation/utils.py:1212: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  "You have modified the pretrained model configuration to control generation. This is a"
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/home/user/anaconda3/envs/language-planner/lib/python3.7/site-packages/transformers/generation/utils.py:1302: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[{'generated_text': 'New way to solve the problem is to give your children a new school that will be similar to yours. The first step is to set up a new school that will teach English and you must send him to a similar school in a different country. The'}]


## 지정해서 사용하기
pipeline = tokenizer + model

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained() # 모델 이름
encoded_input = tokenizer('') # input, padding, truncation, return_tensors='pt', ...

decoding method
* greedy search (default)
* contrastive search - non-repetitive, long
* beam-search decoding - 전체 sequence를 참조해서 가장 prob 높은 것을 고른다, num beams는 keep하는 hypothesis의 수

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'gpt2-xl' # gpt2-xl
tokenizer = AutoTokenizer.from_pretrained(model_name)
inputs = tokenizer('Correct this to standard English:\n slice WineBottle\n', return_tensors='pt')
model = AutoModelForCausalLM.from_pretrained(model_name)
outputs = model.generate(**inputs, num_beams=5, max_new_tokens=50)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Correct this to standard English:\n slice WineBottle\nThis will give you a list of all the WineBottles you have in your inventory. If you have more than one WineBottle in your inventory, you will get a list of all the WineBottles you have in your inventory. If you']

# openai api

앗... 여러개 할 수 있었음

In [2]:
import openai
from utils import config

openai.api_key = config.OPENAI['api_key']
openai.organization = config.OPENAI['organization']

prompt = ["Correct this to standard English:\n\n%s\n"%('abs'), "Correct this to standard English:\n\n%s\n"%('do something'), "Correct this to standard English:\n\n%s\n"%('can you help me out?')]

response = openai.Completion.create(
                model = 'text-davinci-003',
                prompt = prompt,
                temperature = 0,
                logprobs=1,
                n = 1,
                max_tokens = 60,
                stop = None
            )


In [5]:
print([c.text for c in response.choices])

['\nAbs', '\nDo something.', '\nCan you help me?']


In [3]:
openai.Model.list()

<OpenAIObject list at 0x7f41adecbcb0> JSON: {
  "data": [
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1669085501,
          "group": null,
          "id": "modelperm-49FUp5v084tBB49tC4z8LPH5",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "babbage"
    },
    {
      "created": 1649359874,
      "id": "davinci",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sa

# Memo

In [6]:
from utils.plan_module import Plan
import json
import pickle
import os
import pprint

pp = pprint.PrettyPrinter()

plan_data = pickle.load(open('result/alfred/prompt10/vanilla/valid_seen_1.p', 'rb'))
root = 'data/alfred_data/json_2.1.0/valid_seen'
cnt = 0
for cc, goal in enumerate(plan_data):
    best_plan = plan_data[goal]['plan'][0]
    cut_idx = -1
    for i, triplet in enumerate(best_plan):
        if triplet == ['', '', '']:
            cut_idx = i
            break
    if cut_idx > 0:
        best_plan = best_plan[:cut_idx]
    try:
        generated_plan = Plan(high_desc=goal, triplets=best_plan)
    except:
        print(best_plan)
    suc = False
    for ep in os.listdir(root):
        for trial in os.listdir(os.path.join(root, ep)):    
            traj_data = json.load(open(os.path.join(root, ep, trial, 'traj_data.json'), 'r'))
            anns = [ann['task_desc'] for ann in traj_data['turk_annotations']['anns']]
            if goal not in anns:
                continue
            task_type = traj_data['task_type']
            pddl_param = traj_data['pddl_params']
            if cnt == 11:
                print()
            suc = generated_plan.is_executable() and generated_plan.is_plan_fulfilled(task_type, pddl_param)
    print(cc, ':', end=' ')
    print(goal)
    pp.pprint(best_plan)
    print(task_type, pddl_param)
    if not suc:
        print('Failure')
    else:
        print('Success')
    print()
    print('-'*30)
    print()
    cnt += 1
    if cnt > 200:
        break

0 : pick up keys, turn on lamp
[['PickupObject', 'KeyChain', 'CoffeeTable'],
 ['ToggleObject', 'FloorLamp', 'FloorLamp']]
look_at_obj_in_light {'mrecep_target': '', 'object_sliced': False, 'object_target': 'KeyChain', 'parent_target': '', 'toggle_target': 'FloorLamp'}
Success

------------------------------

1 : Look at a set of keys under a lit tall lamp.
[['PickupObject', 'KeyChain', 'SideTable'],
 ['ToggleObject', 'FloorLamp', 'FloorLamp']]
look_at_obj_in_light {'mrecep_target': '', 'object_sliced': False, 'object_target': 'KeyChain', 'parent_target': '', 'toggle_target': 'FloorLamp'}
Success

------------------------------

2 : Turn on the living room lamp.
[['PickupObject', 'Newspaper', 'ArmChair'],
 ['ToggleObject', 'FloorLamp', 'FloorLamp']]
look_at_obj_in_light {'mrecep_target': '', 'object_sliced': False, 'object_target': 'KeyChain', 'parent_target': '', 'toggle_target': 'FloorLamp'}
Failure

------------------------------

3 : Put a pot containing a knife and slice of potato 